In [ ]:
import numpy as np
import torch
import torch.nn.functional as F  #激活函数
from torch import nn
from torch import optim  #优化器
import torchvision
from torchvision.datasets import MNIST  #自带的数据集，手写数字
import matplotlib.pyplot as plt
import torch.utils.data

In [ ]:
class CifarDataset(torch.utils.data.Dataset):
  def __init__(self, root_dir):
    #初始化数据集，包含一系列图片与对应标签
    super().__init__()
    raise NotImplementedError
  
  def __len__(self):
    #返回数据集大小
    raise NotImplementedError
  
  def __getitem__(self, index):
    #返回第index个样本
    raise NotImplementedError
  
  

In [ ]:
train_dataset=CifarDataset(TRAIN_DIRECTORY_PATH)
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
